# Debuggable Execution

It is common to have an output that went through multiple transformations.
One example is an output that depends on multiple machine learning models.
Another example could be a computation that depends on multiple inputs, whose
fetching process can fail and use a fall back value.

It is desirable to understand from a user perspective what are the dependency
and what were the values that returned the final output.

## A Tale of Two Agents
The following attempts to describe such scenario:

There are two agents, Bond and Six, who would go out to recruit warriors from
different villages, Avalon, Gotham and Vice. Individual agent goes through village
sequentially but either agent conduct their recruiting in parallel.

After both agents come back, they go into fight and the one with more recruits won
the game.

We would like to know not only the winner but also the process history, i.e.:

```
"Bond (0) + 6"--|         Avalon--"Six (0) + 3"
   |            \--Gotham           |
"Bond (6) + 2"-|       |----------"Six (3) + 0"
   |           \-Vice--+-----\      |
"Bond (8) + 3"---------/      \---"Six (3) + 10"
          \                        /
           "Bond (11) vs. Six (13)"
                    |
              "Six (2) won!"
```

We want to construct this graph with as little boiler-plate code as possible
such that we can focus on the logic instead of merging of debugging
information.

In [8]:
{-# LANGUAGE TemplateHaskell #-}

import Control.Concurrent
import Control.Concurrent.Async
import Control.Lens
import Control.Monad.Writer.Lazy
import Data.Maybe
import Data.Tree as Tree

We use a Tree data structure to keep track of task dependencies in DAG.

Our requirements are:

1. It has a `Monoid` instance for `Writer`
2. Dependencies can be merged correctly

It is probably reasonable to require each operation must left message via `Node "...message..." []` and `mappend` always prepend dependencies onto this node. 

**This breaks left/right associativity of the monoid.**

> Q: What is the right Monoid abstraction for this log?

In [2]:
data DAGLog a = Node (Maybe a) [DAGLog a] | Leaf a | Empty deriving (Show)  

instance Monoid (DAGLog t) where  
    mempty = Empty
    -- left identify
    mappend Empty a = a
    -- right identity
    mappend a Empty = a
    -- merge into logging node
    mappend a (Node m logs) = Node m (a : logs)
    -- merge any other two nodes are illegal as far as our use case is concerned
    mappend _ _ = error "Must merge into node"
    
nodeMessage :: String -> DAGLog String
nodeMessage m = Node (Just m) []

-- recruited = (gotham `mappend` (bond `mappend` (nodeMessage "Found one!")))
--     where gotham = Leaf "Gotham was found!"
--           bond = Leaf "Bond was born!"
-- recruited

Next we define Villages and Agents respectively. We use Lens to simplify Agent data transformation.

The `recruit` function is a blocking IO action that simulate agent goes into village for recruiting.

In [19]:
type Message = String
data Village = Avalon | Gotham | Vice deriving (Show)
logVillage :: Village -> WriterT (DAGLog String) IO Village
logVillage v = writer (v, Leaf $ show v ++ " was found!")

type Recruits = Int
data Agent = Bond {_r :: Recruits} | Six {_r :: Recruits} deriving Show
logAgent :: Agent -> WriterT (DAGLog String) IO Agent
logAgent v = writer (v, Leaf $ show v ++ " was born!")
makeLenses ''Agent

add :: Agent -> Recruits -> Agent
add agent n = r %~ (+n) $ agent

-- Agent goes into Village to `recruit` more Recruits (blocking process)
recruit :: Agent -> Village -> WriterT (DAGLog String) IO Agent

type Seconds = Int
recruit' :: Agent -> Seconds -> Recruits -> Message -> WriterT (DAGLog String) IO Agent
recruit' a d r m = do
  tell $ nodeMessage m
  lift $ threadDelay $ d * 10 ^ 6
  return $ a `add` r

recruit a@(Bond 0) Gotham = recruit' a 3 6 "Bond knew 6 perfect dudes at Gotham."
recruit a@(Bond _) Gotham = recruit' a 2 3 "Only got 3 this time."
recruit a@(Bond _) Vice = recruit' a 1 2 "Bond fetched 2 agents in just a second!"

recruit a@(Six _) Avalon = recruit' a 1 3 "3 ==> 6!"
recruit a@(Six _) Vice = recruit' a 2 10 "Number Six snatched 10 agents swiftly."

recruit a _ = recruit' a 0 0 "No one was willing to join the cause. Sadness..."

Combat is a simple function that compares two agent and declare the one with more recruits the winner.

In [21]:
combat :: Agent -> Agent -> WriterT (DAGLog String) IO (Maybe Agent)
combat agentX agentY = case compare x y of
    LT -> do
      lift $ threadDelay $ x * 10 ^ 5
      tell (nodeMessage $ show winnerY ++ " won!")
      return $ Just winnerY
    GT -> do
      lift $ threadDelay $ y * 10 ^ 5
      tell (nodeMessage $ show winnerX ++ " won!")
      return $ Just winnerX
    EQ -> do
      lift $ threadDelay $ x * 10 ^ 5
      tell (nodeMessage "It was a tie!")
      return Nothing
    where x = agentX^.r
          y = agentY^.r
          winnerX = agentX `add` (-y)
          winnerY = agentY `add` (-x)

Now we can take this for a swirl. We replicate the tree in the example. For the most part, this seems successful in a sense that we can express logic without worrying too much about logging.

It is unsatisfactory that we have to nest operations to create the right boundary. The nesting is also in the wrong direction:

```haskell
recruit3 (recruit2 (recruit1 agent))
```
instead of 
```haskell
agent |> recruit1 |> recruit2 |> recruit3
```

Note we can throw in `async` to agents to fulfill our requirement of concurrency quite easily. It is omitted here for simplicity.

In [22]:
bondW :: WriterT (DAGLog String) IO Agent
bondW = 
    (do
        bond <- logAgent $ Bond 0
        gotham <- gothamW
        bond `recruit` gotham) >>=
    (\bond -> do
        vice <- viceW
        bond `recruit` vice) >>=
    (\bond -> do
        gotham <- gothamW
        bond `recruit` gotham)
  where avalonW = logVillage Avalon
        gothamW = logVillage Gotham
        viceW = logVillage Vice

sixW :: WriterT (DAGLog String) IO Agent
sixW =
    (do
        six <- logAgent $ Six 0
        avalon <- avalonW
        six `recruit` avalon) >>=
    (\six -> do
        gotham <- gothamW
        six `recruit` gotham) >>=
    (\six -> do
        vice <- viceW
        six `recruit` vice)
  where avalonW = logVillage Avalon
        gothamW = logVillage Gotham
        viceW = logVillage Vice

rec1 :: WriterT (DAGLog String) IO (Maybe Agent)
rec1 = do
  bond <- bondW
  six <- sixW
  bond `combat` six
fmap fst $ runWriterT rec1

Just (Six {_r = 2})

We transform our `DAGLog` to a `Tree` for visualization:

In [9]:
toTree :: Monoid a => DAGLog a -> Tree.Tree a
toTree (Leaf a) = Tree.Node a []
toTree (Node m l) = Tree.Node (fromMaybe mempty m) (map toTree l)

printTree :: (WriterT (DAGLog String) IO a) -> IO ()
printTree w = do
    res <- runWriterT w
    putStrLn $ drawTree $ toTree $ snd res

In [10]:
printTree rec1

Six {_r = 2} won!
|
+- Only got 3 this time.
|  |
|  +- Bond fetched 2 agents in just a second!
|  |  |
|  |  +- Bond knew 6 perfect dudes at Gotham.
|  |  |  |
|  |  |  +- Bond {_r = 0} was born!
|  |  |  |
|  |  |  `- Gotham was found!
|  |  |
|  |  `- Vice was found!
|  |
|  `- Gotham was found!
|
`- Number Six snatched 10 agents swiftly.
   |
   +- No one was willing to join the cause. Sadness...
   |  |
   |  +- 3 ==> 6!
   |  |  |
   |  |  +- Six {_r = 0} was born!
   |  |  |
   |  |  `- Avalon was found!
   |  |
   |  `- Gotham was found!
   |
   `- Vice was found!

In [25]:
1

1